In [1]:
import os
import numpy as np
import pandas as pd
import csv
import json
from keras.layers import Input, LSTM, Dense
from keras.models import Model
import numpy as np

Using TensorFlow backend.


In [2]:
path = os.path.join('mot_benchmark', 'train')

In [3]:
with open('mot_benchmark.json') as f:
    data = json.load(f)

In [4]:
max_fps = 30
max_height = 1080
max_width = 1920

### Create the dataset of tracks

In [ ]:
all_tracks = {}
file_names = None
file_count = 0
for subdir, dirs, files in os.walk(path):
    if file_names is None: file_names = dirs
    if 'gt' in subdir:
        print(subdir)
        for file in files:
            if 'gt.txt' == file:
                file_path = os.path.join(subdir, file)
                df = pd.read_csv(file_path, names=['frame', 'id', 'bb_left', 'bb_top', \
                                                   'bb_width', 'bb_height', 'conf', 'x', 'y', 'z'])
                
                fps = data[file_names[file_count]]['fps']
                height = data[file_names[file_count]]['height']
                width = data[file_names[file_count]]['width']
                
                file_tracks = []
                for id_num in range(1, df['id'].max()+1):
                    track_df = df[df['id'] == id_num]
                    track = track_df.values[:, np.r_[0:1,2:6]]
                    
                    if len(track) < 3:  # reject tracks with fewer than three detections
                        continue
                    
                    last_frame_num = 0  # reject track with missing frames
                    for i, frame in enumerate(track):
                        if last_frame_num == 0:
                            last_frame_num = frame[0]
                            continue
                        else:
                            if frame[0] != last_frame_num +1: 
                                break
                            last_frame_num = frame[0]
                            
                    print(track)
                    break
                    
                    # good track - now normalize based on h & w of image, append fps and add to the list
                    np.array([track[:,1] / float(width), track[:,2] / float(height), 
                                                 track[:,3] / float(width), track[:,4] / float(height),
                                                 [fps/max_fps for i in range(len(track))],
                                                 [width/max_width for i in range(len(track))],
                                                 [height/max_height for i in range(len(track))]])
                    
                    file_tracks.append(normalized_track.T)
                all_tracks[file_names[file_count]] = file_tracks
                file_count +=1

### Create dataset

In [6]:
examples = []
labels = []

n_tracks = 0
for video in all_tracks.keys():
    track_list = all_tracks[video]
    for track in track_list:
        for i in range(1, len(track)-1):
            examples.append(track[:i])
            labels.append(track[i][:-3])

examples = np.array(examples)
labels = np.array(labels)

In [7]:
print(len(examples), len(labels))

42038 42038


In [8]:
inputs = Input(shape=(None, 7), name='inputs')
x = LSTM(32)(inputs)
outputs = Dense(4)(x)

model = Model(inputs=inputs, outputs=outputs)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, None, 7)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                5120      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 132       
Total params: 5,252
Trainable params: 5,252
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [10]:
for i in range(2): # 100 epochs
    
    training_loss = 0
    for example, label in zip(examples[:1000], labels[:1000]):
        hist_obj = model.fit(np.expand_dims(example, 0), np.expand_dims(label, 0), verbose=0)
        training_loss += hist_obj.history['loss'][0]
    
    validation_loss = 0
    for example, label in zip(examples[1000:1500], labels[1000:1500]):
        val_loss = model.evaluate(np.expand_dims(example, 0), np.expand_dims(label, 0), verbose=0)
        validation_loss += val_loss

    print('epoch', i, ': training loss:', training_loss/float(100), 'validation loss:', validation_loss/float(100))

epoch 0 : training loss: 0.03210819611310924 validation loss: 0.1424982517096214
epoch 1 : training loss: 0.02235694718614198 validation loss: 0.11238813692291842


In [11]:
model.save('PETS09-S2L1.h5')

In [16]:
model.predict(np.expand_dims(examples[1084], 0))

array([[ 0.69293594,  0.49949664, -0.00287014,  0.09591044]],
      dtype=float32)

In [10]:
test = labels[1084]

In [11]:
test[0]

0.5494791666666666